In [1]:
import random
import numpy as np 
import random
import matplotlib.pyplot as plt
import time 
from collections import deque  
from ipywidgets import IntProgress
from IPython.display import display
import statistics
random.seed(a=3)

In [20]:
def generate_tsp_incomplete(num_sommets, max_poids_arete, max_intervalle_inf, max_intervalle_sup):
    # Initialisation de la matrice de poids avec des valeurs infinies
    matrice = [[float('inf') for _ in range(num_sommets)] for _ in range(num_sommets)]
    
    intervalles = {}
    for i in range(num_sommets):
        intervalle_start = random.randint(0, max_intervalle_inf)
        intervalle_end = intervalle_start + random.randint(max_intervalle_inf, max_intervalle_sup)
        intervalles[i] = (intervalle_start, intervalle_end) 
    
    # Utiliser Prim's algorithm pour créer un arbre couvrant minimal (MST)
    connected = set([0])
    edges = []

    while len(connected) < num_sommets:
        min_edge = (None, None, float('inf'))
        for u in connected:
            for v in range(num_sommets):
                if v not in connected and u != v:
                    poids = random.randint(1, max_poids_arete)
                    if poids < min_edge[2]:
                        min_edge = (u, v, poids)
        u, v, poids = min_edge
        if u is not None and v is not None:
            matrice[u][v] = poids
            matrice[v][u] = poids
            connected.add(v)
            edges.append((u, v, poids))

    # Ajouter des arêtes supplémentaires de manière aléatoire
    for i in range(num_sommets):
        for j in range(i + 1, num_sommets):
            if matrice[i][j] == float('inf') and random.choice([True, False]):
                poids = random.randint(1, max_poids_arete)
                matrice[i][j] = poids
                matrice[j][i] = poids

    return np.array(matrice), intervalles

In [24]:
num_sommets = 5
max_poids_arete = 10
max_intervalle_inf = 20
max_intervalle_sup = 100

# Générer les données du problème
tsp_matrice, tsp_intervalles = generate_tsp_incomplete(num_sommets, max_poids_arete, max_intervalle_inf, max_intervalle_sup)

# Afficher la matrice pondérée
print("\nMatrice pondérée:")
for row in tsp_matrice:
    print(row)
    
# Afficher les intervalles
print("\nIntervalles des sommets")
print(tsp_intervalles)


Matrice pondérée:
[inf  1.  1.  3.  1.]
[ 1. inf inf  1. inf]
[ 1. inf inf  5.  2.]
[ 3.  1.  5. inf inf]
[ 1. inf  2. inf inf]

Intervalles des sommets
{0: (6, 79), 1: (19, 87), 2: (7, 64), 3: (7, 49), 4: (16, 111)}


In [4]:
# Paramètres de la colonie de fourmis
num_ants = 10
num_iterations = 100
alpha = 1.0
beta = 2.0
evaporation_rate = 0.5
pheromone_constant = 100

In [26]:
def is_edge_valid(from_node, to_node, current_length, visited, num_vertice_taken):
    weight = tsp_matrice[from_node][to_node]
    if weight == np.inf or ((from_node, to_node) in num_vertice_taken and num_vertice_taken[(from_node, to_node)] > num_sommets):
        return (False, 0)
    if not (to_node in visited):
        min_weight, max_weight = tsp_intervalles[to_node]
        if current_length + weight > max_weight:
            return (False, 0)
        if current_length + weight < min_weight:
            return (True,  (min_weight - current_length))
    return (True, weight)

In [6]:

def update_pheromones(best_path, best_path_length, pheromone_matrix):
    pheromone_matrix *= (1 - evaporation_rate)
    for i in range(len(best_path) - 1):
        from_node = best_path[i]
        to_node = best_path[i + 1]
        pheromone_matrix[from_node][to_node] += pheromone_constant / best_path_length

In [7]:
def select_next_node(current_node, visited, current_length, pheromone_matrix, num_vertice_taken):
    probabilities = []
    total_prob = 0
    pheromone_row = pheromone_matrix[current_node]
    heuristic_row = (1.0 / tsp_matrice[current_node]) ** beta

    for node in range(num_sommets):
        if node != current_node:
            valide, poids = is_edge_valid(current_node, node, current_length, visited, num_vertice_taken)
            if valide:
                pheromone_level = pheromone_row[node] ** alpha
                heuristic_level = heuristic_row[node]
                probability = pheromone_level * heuristic_level
                probabilities.append((node, probability))
                total_prob += probability
    
    if total_prob == 0:
        return (None, 0)
    
    threshold = random.uniform(0, total_prob)
    cumulative_prob = 0
    for node, probability in probabilities:
        cumulative_prob += probability
        if cumulative_prob >= threshold:
            valide, poids = is_edge_valid(current_node, node, current_length, visited, num_vertice_taken)
            return (node, poids)
    return (None, 0)


In [8]:
def ant_colony_optimization():
    best_path = None
    best_path_length = float('inf') 
    path_lengths = deque(())
    best_path_lengths = deque(())
    execution_times = [] 
    distances = [] 
    
    # Initialiser les niveaux de phéromones uniquement sur les arêtes
    pheromone_matrix = np.ones((num_sommets, num_sommets))

    for iteration in range(num_iterations):
        start_time = time.time() 
        all_paths = []
        for ant in range(num_ants):
            current_node = random.randint(0, num_sommets - 1)
            path = [current_node]
            visited = {current_node}
            min_weight, max_weight = tsp_intervalles[current_node]
            current_length = min_weight
            num_vertice_taken = {}
            
            # Construire un chemin
            while len(visited) < num_sommets:
                next_node, weight_to_add = select_next_node(current_node, visited, current_length, pheromone_matrix, num_vertice_taken)
                if next_node is None:
                    break
                current_length += weight_to_add
                current_node = next_node
                path.append(current_node)
                visited.add(current_node)
                if (current_node, next_node) in num_vertice_taken:
                    num_vertice_taken[(current_node, next_node)] += 1
                else: 
                    num_vertice_taken[(current_node, next_node)] = 1
            
            # Vérifier si un cycle est formé
            valid, weight = is_edge_valid(current_node, path[0], current_length, visited, num_vertice_taken)
            if len(visited) == num_sommets and valid:
                current_length += weight
                path_lengths.append(current_length)
                path.append(path[0]) 
                all_paths.append((path, current_length))
                if current_length < best_path_length:
                    best_path_length = current_length
                    best_path = path
                best_path_lengths.append(best_path_length)    
            
            
        # Mettre à jour les phéromones uniquement sur le meilleur chemin trouvé
        if best_path:
            update_pheromones(best_path, best_path_length, pheromone_matrix)
        end_time = time.time()
        execution_times.append(end_time - start_time)
        distances.append(best_path_length) 
    
    return best_path, best_path_length, path_lengths, best_path_lengths , execution_times, distances   

In [9]:
#Ne pas utiliser, ne donne pas la borne inf
def borne_inferieure(tsp_matrice):
    # Nombre de villes
    num_cities = tsp_matrice.shape[0]

    # Création du problème de minimisation
    prob = pulp.LpProblem("tsp", pulp.LpMinimize)

    # Variables de décision
    x = pulp.LpVariable.dicts("x", ((i, j) for i in range(num_cities) for j in range(num_cities)), cat='Binary')

    # Fonction objectif : minimiser la distance totale
    prob += pulp.lpSum(tsp_matrice[i][j] * x[i, j] for i in range(num_cities) for j in range(num_cities) if tsp_matrice[i][j] != np.inf)

    # Contraintes
    for i in range(num_cities):
        # Chaque ville doit être quittée au moins une fois sup ou egale 1 
        prob += pulp.lpSum(x[i, j] for j in range(num_cities) if i != j and tsp_matrice[i][j] != np.inf) == 1
        # Chaque ville doit être entrée au moins une fois
        prob += pulp.lpSum(x[j, i] for j in range(num_cities) if i != j and tsp_matrice[j][i] != np.inf) == 1

    # Elimination des sous-tours
    u = pulp.LpVariable.dicts("u", (i for i in range(num_cities)), lowBound=0, upBound=num_cities-1, cat='Continuous')

    for i in range(1, num_cities):
        for j in range(1, num_cities):
            if i != j and tsp_matrice[i][j] != np.inf:
                prob += u[i] - u[j] + (num_cities - 1) * x[i, j] <= num_cities - 2

    # Résolution du problème
    prob.solve()

    if pulp.LpStatus[prob.status] == "Optimal":
        solution = [(i, j) for i in range(num_cities) for j in range(num_cities) if pulp.value(x[i, j]) == 1]
        return pulp.value(prob.objective), solution
    else:
        return None, None 


In [10]:
def plot_route(path, points, matrice):
    plt.figure(figsize=(10, 5))
    plt.scatter(points[:, 0], points[:, 1], c='blue')

    # Highlight the start node
    start_node = path[0]
    plt.scatter(points[start_node, 0], points[start_node, 1], c='red', label='Start Node')

    # Annotate nodes with their indices
    for idx, point in enumerate(points):
        plt.text(point[0], point[1], str(idx), fontsize=12, ha='right', va='bottom')

    for i in range(len(path) - 1):
        from_node = path[i]
        to_node = path[i + 1]
        plt.plot([points[from_node, 0], points[to_node, 0]], 
                 [points[from_node, 1], points[to_node, 1]], 'k-')
        # Add weights (distances) on the edges
        mid_point = (points[from_node] + points[to_node]) / 2
        weight = matrice[from_node][to_node]
        plt.text(mid_point[0] + 2, mid_point[1] + 2, f'{weight:.2f}', fontsize=9, color='green')

    # Add the weight for the edge returning to the start node
    from_node = path[-1]
    to_node = path[0]
    plt.plot([points[from_node, 0], points[to_node, 0]], 
             [points[from_node, 1], points[to_node, 1]], 'k-')
    mid_point = (points[from_node] + points[to_node]) / 2
    weight = matrice[from_node][to_node]
    plt.text(mid_point[0] + 2, mid_point[1] + 2, f'{weight:.2f}', fontsize=9, color='green')

    plt.title('Best Route Found')
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.legend()
    plt.show() 

In [11]:
def plot_routeLinear(solution, points):
    if solution:
        route = np.array([points[i] for i, j in solution] + [points[solution[0][0]]])
        plt.figure(figsize=(10, 6))
        plt.plot(route[:, 0], route[:, 1], 'o-', label='Route')
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.title('TSP Route')
        plt.legend()
        plt.show()
    else:
        print("No solution to plot.")  

In [12]:
def plot_performance(execution_times, distances):
    fig, ax1 = plt.subplots()

    color = 'tab:red'
    ax1.set_xlabel('Iteration')
    ax1.set_ylabel('Execution Time (s)', color=color)
    ax1.plot(execution_times, color=color)
    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
    color = 'tab:blue'
    ax2.set_ylabel('Best Path Length', color=color)  
    ax2.plot(distances, color=color)
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()   
    plt.title('Performance Over Iterations')
    plt.show() 

In [13]:
def plot_quality(best_path_lengths):
    plt.figure(figsize=(10, 5))
    plt.plot(best_path_lengths, marker='o', linestyle='-', color='b')
    plt.title('ACO Performance: Best Path Length Over Iterations')
    plt.xlabel('Iteration')
    plt.ylabel('Best Path Length')
    plt.grid(True)
    plt.show()


In [14]:
def plot_vi(path_lengths, best_path_lengths):
    plt.figure(figsize=(10, 5))
    plt.xlabel("Nombre d'itérations", fontsize=16)
    plt.ylabel("Valeur", fontsize=16) 
    plt.plot(range(len(path_lengths)), path_lengths, label='Longueur du chemin courant')
    plt.plot(range(len(best_path_lengths)), best_path_lengths, label='Meilleure longueur du chemin', linestyle='--')
    plt.legend()
    plt.title("Évolution des longueurs de chemin au cours des itérations")
    plt.grid(True)
    plt.show() 

In [15]:
best_path, best_path_length, path_lengths, best_path_lengths, execution_times, distances = ant_colony_optimization()  
# Generate random points for plotting (for visualization purposes only)
points = np.random.rand(num_sommets, 2) * 100
print(f"Meilleur chemin trouvé : {best_path} avec une longueur de {best_path_length:.2f}")  
# Plot the best route
if best_path:
    plot_route(best_path, points, tsp_matrice)  
    plot_vi(path_lengths, best_path_lengths) 
# Plot the performance
#plot_performance(execution_times, distances)  
#plot_quality(best_path_lengths)  
print(f'Meilleur chemin trouvé: {best_path_length:.2f}')      

KeyboardInterrupt: 

In [16]:
#Parametres de test
nb_test = 100 
matrices_test = {}
intervalles_test = {}

#Creation des instances de test
for i in range(nb_test):
    matrice, intervalle = generate_tsp_incomplete(num_sommets, max_poids_arete, max_intervalle_inf, max_intervalle_sup)
    matrices_test[i] = matrice
    intervalles_test[i] = intervalle
    
print(matrices_test[0], matrices_test[1])

[[inf inf  1.  1. inf]
 [inf inf 10.  4.  9.]
 [ 1. 10. inf inf  7.]
 [ 1.  4. inf inf  3.]
 [inf  9.  7.  3. inf]] [[inf  5.  2.  1.  2.]
 [ 5. inf  4.  6.  2.]
 [ 2.  4. inf 10.  4.]
 [ 1.  6. 10. inf inf]
 [ 2.  2.  4. inf inf]]


In [17]:
def plot_evaporation_test():
    # paramètres du test
    evaporation_min = 0
    evaporation_max = 1

    # on affiche la barre de progression
    nb_steps_bar = 10*nb_test 
    bar = IntProgress(min=0, max=nb_steps_bar, layout={"width" : "100%"})
    display(bar)

    # pour stocker les résultats
    moyennes   = []
    deviations = []
    num_reussites = []

    random.seed(a=3)

    evaporation_rate = evaporation_min

    # cette fois on boucle sur la taille de la liste tabou
    while evaporation_rate <= evaporation_max:            
        bornes = deque(())
        i = 0
        reussites = 0
        while i < nb_test:                                                      
            tsp_matrice = matrices_test[i]
            best_path, best_path_length, path_lengths, best_path_lengths = ant_colony_optimization()
            if best_path_length != np.inf:
                reussites +=1
            bornes.append(best_path_length)
            #else:
                #bornes.append(1000)
            bar.value += 1
            i +=1
        evaporation_rate += 0.1

        num_reussites.append(reussites)
        moyennes.append(statistics.fmean(bornes))               
        deviations.append(np.std(bornes))                       

    # on cache la barre de progression
    bar.close()

    # Créer une échelle d'abscisse normalisée de 0 à 1
    x_values = [i / (len(moyennes) - 1) for i in range(len(moyennes))]

    # affichage de la courbe de moyenne
    plt.plot(x_values, moyennes)

    # affichage de la bande d'écart-type
    plt.fill_between(x_values,
                     np.subtract(moyennes, deviations), # borne haute
                     np.add(moyennes, deviations),      # borne basse
                     alpha=.1)                          # transparence
    plt.xlabel("évaporation")
    plt.ylabel("distance moyenne trouvée")
    plt.title("Impact de l'évaporation sur la qualité des solutions")
    plt.show()

    num_reussites

    plt.plot(x_values, num_reussites)
    plt.show()

In [18]:
plot_evaporation_test()

IntProgress(value=0, layout=Layout(width='100%'), max=1000)

KeyboardInterrupt: 

In [ ]:
ant_colony_optimization()